In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, auc, 
                             precision_recall_curve, average_precision_score)
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [41]:
def evaluate_quick(model, X_test, y_test, model_name="Model"):
    """Tez baholash"""
    y_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)
    
    # Optimal threshold topish
    thresholds = np.arange(0.1, 0.9, 0.01)
    costs = []
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total_cost = (fn * 10000) + (fp * 500)
        costs.append(total_cost)
    
    optimal_threshold = thresholds[np.argmin(costs)]
    y_pred_optimal = (y_proba >= optimal_threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()
    min_cost = (fn * 10000) + (fp * 500)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n{'='*70}")
    print(f"📊 {model_name}")
    print(f"{'='*70}")
    print(f"ROC-AUC:     {roc_auc:.4f}")
    print(f"PR-AUC:      {pr_auc:.4f}")
    print(f"Precision:   {precision:.3f}")
    print(f"Recall:      {recall:.3f}")
    print(f"F1-Score:    {f1:.3f}")
    print(f"Threshold:   {optimal_threshold:.3f}")
    print(f"FN Count:    {fn}")
    print(f"Total Cost:  ${min_cost:,}")
    
    return {
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'fn_count': fn,
        'total_cost': min_cost,
        'threshold': optimal_threshold,
        'proba': y_proba
    }


In [42]:
X = pd.read_csv("../merge/merged_x.csv")
y = pd.read_csv("../merge/merged_y.csv")  

print(X.shape, y.shape)
print("\n" + "🚀 "*35)
print("BOSQICH 1: ENSEMBLE MODELLAR O'RGATISH")
print("🚀 "*35)

(89999, 56) (89999, 1)

🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 
BOSQICH 1: ENSEMBLE MODELLAR O'RGATISH
🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 


In [43]:
print("\n📦 Ma'lumotlarni bo'lish...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

print(f"✓ Train: {X_train_split.shape}, Val: {X_val.shape}, Test: {X_test.shape}")



📦 Ma'lumotlarni bo'lish...
✓ Train: (57599, 56), Val: (14400, 56), Test: (18000, 56)


In [44]:
print("\n" + "="*70)
print("1️⃣  CATBOOST MODEL")
print("="*70)

catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    class_weights=[1, 20],
    random_seed=42,
    verbose=False,
    early_stopping_rounds=50
)

catboost_model.fit(
    X_train_split, y_train_split,
    eval_set=(X_val, y_val),
    verbose=False
)

catboost_metrics = evaluate_quick(catboost_model, X_val, y_val, "CatBoost")



1️⃣  CATBOOST MODEL

📊 CatBoost
ROC-AUC:     0.7883
PR-AUC:      0.2250
Precision:   0.110
Recall:      0.782
F1-Score:    0.192
Threshold:   0.420
FN Count:    160
Total Cost:  $3,933,000


In [45]:
print("\n" + "="*70)
print("2️⃣  LIGHTGBM MODEL")
print("="*70)

lgbm_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight={0: 1, 1: 20},
    random_state=42,
    early_stopping_rounds=100,
    verbose=-1
)

lgbm_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)]
)

lgbm_metrics = evaluate_quick(lgbm_model, X_val, y_val, "LightGBM")



2️⃣  LIGHTGBM MODEL

📊 LightGBM
ROC-AUC:     0.7706
PR-AUC:      0.2068
Precision:   0.125
Recall:      0.675
F1-Score:    0.211
Threshold:   0.350
FN Count:    239
Total Cost:  $4,127,000


In [46]:
print("\n" + "="*70)
print("3️⃣  XGBOOST MODEL")
print("="*70)

xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=20,
    random_state=42,
    eval_metric='auc',
    early_stopping_rounds=50,
    verbosity=0
)

xgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=False
)

xgb_metrics = evaluate_quick(xgb_model, X_val, y_val, "XGBoost")


3️⃣  XGBOOST MODEL

📊 XGBoost
ROC-AUC:     0.7769
PR-AUC:      0.2128
Precision:   0.107
Recall:      0.747
F1-Score:    0.187
Threshold:   0.410
FN Count:    186
Total Cost:  $4,152,000


In [47]:
print("\n" + "="*70)
print("4️⃣  SIMPLE AVERAGING ENSEMBLE")
print("="*70)

def simple_ensemble(models, X):
    """O'rtacha proba"""
    predictions = []
    for model in models:
        predictions.append(model.predict_proba(X)[:, 1])
    return np.mean(predictions, axis=0)

models_list = [catboost_model, lgbm_model, xgb_model]
y_proba_avg = simple_ensemble(models_list, X_val)

# Baholash
roc_auc = roc_auc_score(y_val, y_proba_avg)
pr_auc = average_precision_score(y_val, y_proba_avg)

# Optimal threshold
thresholds = np.arange(0.1, 0.9, 0.01)
costs = []
for threshold in thresholds:
    y_pred = (y_proba_avg >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    total_cost = (fn * 10000) + (fp * 500)
    costs.append(total_cost)

optimal_threshold = thresholds[np.argmin(costs)]
y_pred_optimal = (y_proba_avg >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred_optimal).ravel()
min_cost = (fn * 10000) + (fp * 500)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"ROC-AUC:     {roc_auc:.4f}")
print(f"PR-AUC:      {pr_auc:.4f}")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"F1-Score:    {f1:.3f}")
print(f"FN Count:    {fn}")
print(f"Total Cost:  ${min_cost:,}")

ensemble_avg_metrics = {
    'roc_auc': roc_auc,
    'pr_auc': pr_auc,
    'f1': f1,
    'fn_count': fn,
    'total_cost': min_cost
}



4️⃣  SIMPLE AVERAGING ENSEMBLE
ROC-AUC:     0.7862
PR-AUC:      0.2206
Precision:   0.125
Recall:      0.705
F1-Score:    0.212
FN Count:    217
Total Cost:  $3,983,500


In [48]:
print("\n" + "="*70)
print("5️⃣  WEIGHTED ENSEMBLE (OPTIMIZED)")
print("="*70)

from scipy.optimize import minimize

def objective(weights):
    """Optimal weights topish uchun objective function"""
    proba = np.average([
        catboost_model.predict_proba(X_val)[:, 1],
        lgbm_model.predict_proba(X_val)[:, 1],
        xgb_model.predict_proba(X_val)[:, 1]
    ], axis=0, weights=weights)
    
    return -roc_auc_score(y_val, proba)  # Maximize AUC

# Constraints va bounds
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1) for _ in range(3)]
initial_weights = [1/3, 1/3, 1/3]

result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights = result.x

print(f"✓ Optimal weights:")
print(f"  CatBoost: {optimal_weights[0]:.3f}")
print(f"  LightGBM: {optimal_weights[1]:.3f}")
print(f"  XGBoost:  {optimal_weights[2]:.3f}")

# Weighted ensemble prediction
y_proba_weighted = np.average([
    catboost_model.predict_proba(X_val)[:, 1],
    lgbm_model.predict_proba(X_val)[:, 1],
    xgb_model.predict_proba(X_val)[:, 1]
], axis=0, weights=optimal_weights)
# Baholash
roc_auc = roc_auc_score(y_val, y_proba_weighted)
pr_auc = average_precision_score(y_val, y_proba_weighted)

# Optimal threshold
costs = []
for threshold in thresholds:
    y_pred = (y_proba_weighted >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    total_cost = (fn * 10000) + (fp * 500)
    costs.append(total_cost)

optimal_threshold = thresholds[np.argmin(costs)]
y_pred_optimal = (y_proba_weighted >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred_optimal).ravel()
min_cost = (fn * 10000) + (fp * 500)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nROC-AUC:     {roc_auc:.4f}")
print(f"PR-AUC:      {pr_auc:.4f}")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"F1-Score:    {f1:.3f}")
print(f"FN Count:    {fn}")
print(f"Total Cost:  ${min_cost:,}")

ensemble_weighted_metrics = {
    'roc_auc': roc_auc,
    'pr_auc': pr_auc,
    'f1': f1,
    'fn_count': fn,
    'total_cost': min_cost
}



5️⃣  WEIGHTED ENSEMBLE (OPTIMIZED)
✓ Optimal weights:
  CatBoost: 0.333
  LightGBM: 0.333
  XGBoost:  0.333

ROC-AUC:     0.7862
PR-AUC:      0.2206
Precision:   0.125
Recall:      0.705
F1-Score:    0.212
FN Count:    217
Total Cost:  $3,983,500


In [49]:
print("\n" + "="*70)
print("6️⃣  STACKING ENSEMBLE")
print("="*70)

# Base models predictions
meta_features_train = np.column_stack([
    catboost_model.predict_proba(X_train_split)[:, 1],
    lgbm_model.predict_proba(X_train_split)[:, 1],
    xgb_model.predict_proba(X_train_split)[:, 1]
])

meta_features_val = np.column_stack([
    catboost_model.predict_proba(X_val)[:, 1],
    lgbm_model.predict_proba(X_val)[:, 1],
    xgb_model.predict_proba(X_val)[:, 1]
])

# Meta-learner
meta_learner = LogisticRegression(class_weight={0: 1, 1: 20}, random_state=42, max_iter=1000)
meta_learner.fit(meta_features_train, y_train_split)

# Stacking prediction
y_proba_stacked = meta_learner.predict_proba(meta_features_val)[:, 1]

# Baholash
roc_auc = roc_auc_score(y_val, y_proba_stacked)
pr_auc = average_precision_score(y_val, y_proba_stacked)

# Optimal threshold
costs = []
for threshold in thresholds:
    y_pred = (y_proba_stacked >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    total_cost = (fn * 10000) + (fp * 500)
    costs.append(total_cost)

optimal_threshold = thresholds[np.argmin(costs)]
y_pred_optimal = (y_proba_stacked >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred_optimal).ravel()
min_cost = (fn * 10000) + (fp * 500)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"ROC-AUC:     {roc_auc:.4f}")
print(f"PR-AUC:      {pr_auc:.4f}")
print(f"Precision:   {precision:.3f}")
print(f"Recall:      {recall:.3f}")
print(f"F1-Score:    {f1:.3f}")
print(f"FN Count:    {fn}")
print(f"Total Cost:  ${min_cost:,}")

ensemble_stacking_metrics = {
    'roc_auc': roc_auc,
    'pr_auc': pr_auc,
    'f1': f1,
    'fn_count': fn,
    'total_cost': min_cost
}


6️⃣  STACKING ENSEMBLE
ROC-AUC:     0.6799
PR-AUC:      0.1378
Precision:   0.140
Recall:      0.422
F1-Score:    0.210
FN Count:    425
Total Cost:  $5,201,000


In [50]:
print("\n" + "🏆 "*35)
print("BOSQICH 1: YAKUNIY TAQQOSLASH")
print("🏆 "*35)

comparison_data = {
    'Model': ['CatBoost', 'LightGBM', 'XGBoost', 
              'Avg Ensemble', 'Weighted Ensemble', 'Stacking'],
    'ROC-AUC': [
        catboost_metrics['roc_auc'],
        lgbm_metrics['roc_auc'],
        xgb_metrics['roc_auc'],
        ensemble_avg_metrics['roc_auc'],
        ensemble_weighted_metrics['roc_auc'],
        ensemble_stacking_metrics['roc_auc']
    ],
    'PR-AUC': [
        catboost_metrics['pr_auc'],
        lgbm_metrics['pr_auc'],
        xgb_metrics['pr_auc'],
        ensemble_avg_metrics['pr_auc'],
        ensemble_weighted_metrics['pr_auc'],
        ensemble_stacking_metrics['pr_auc']
    ],
    'F1-Score': [
        catboost_metrics['f1'],
        lgbm_metrics['f1'],
        xgb_metrics['f1'],
        ensemble_avg_metrics['f1'],
        ensemble_weighted_metrics['f1'],
        ensemble_stacking_metrics['f1']
    ],
    'FN Count': [
        catboost_metrics['fn_count'],
        lgbm_metrics['fn_count'],
        xgb_metrics['fn_count'],
        ensemble_avg_metrics['fn_count'],
        ensemble_weighted_metrics['fn_count'],
        ensemble_stacking_metrics['fn_count']
    ],
    'Total Cost': [
        f"${catboost_metrics['total_cost']:,}",
        f"${lgbm_metrics['total_cost']:,}",
        f"${xgb_metrics['total_cost']:,}",
        f"${ensemble_avg_metrics['total_cost']:,}",
        f"${ensemble_weighted_metrics['total_cost']:,}",
        f"${ensemble_stacking_metrics['total_cost']:,}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n" + df_comparison.to_string(index=False))

# Eng yaxshi model
best_idx = np.argmax(df_comparison['ROC-AUC'])
print(f"\n🥇 ENG YAXSHI MODEL (ROC-AUC): {df_comparison.iloc[best_idx]['Model']}")
print(f"   ROC-AUC: {df_comparison.iloc[best_idx]['ROC-AUC']:.4f}")

print("\n" + "✅ "*35)
print("BOSQICH 1 YAKUNLANDI! Keyingi: FEATURE ENGINEERING")
print("✅ "*35)


🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 
BOSQICH 1: YAKUNIY TAQQOSLASH
🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 🏆 

            Model  ROC-AUC   PR-AUC  F1-Score  FN Count Total Cost
         CatBoost 0.788286 0.225006  0.192436       160 $3,933,000
         LightGBM 0.770569 0.206787  0.210840       239 $4,127,000
          XGBoost 0.776884 0.212814  0.187117       186 $4,152,000
     Avg Ensemble 0.786189 0.220573  0.212295       217 $3,983,500
Weighted Ensemble 0.786189 0.220573  0.212295       217 $3,983,500
         Stacking 0.679934 0.137752  0.210383       425 $5,201,000

🥇 ENG YAXSHI MODEL (ROC-AUC): CatBoost
   ROC-AUC: 0.7883

✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 
BOSQICH 1 YAKUNLANDI! Keyingi: FEATURE ENGINEERING
✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 
